In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')
METADATAPATH = os.path.join(PATH, 'datasets/metadata')

# load data

In [ ]:
os.listdir(os.path.join(METADATAPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
dataset = mf.data.MFDataset(os.path.join(METADATAPATH,'211112_84_25_2cm_meta.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

# load meta data

In [ ]:
metadata = pd.DataFrame(dataset.metadata)

#dataset.metadata
#test_metadata = pd.DataFrame(test_data.metadata)

In [ ]:
energy_array = metadata['energy'].array
angle_array = metadata['theta_min'].array

#test_energies = np.array(test_metadata['energy'].array).squeeze()
#test_angles = np.array(test_metadata['theta_min'].array).squeeze()

unique_energy = np.sort(np.unique(metadata['energy'].array))
unique_angle = np.sort(np.unique(metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorting_indexes = []

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    sorting_indexes.append(metadata[(metadata['energy'] == pair[0]) & (metadata['theta_min'] == pair[1])].index[0])
    
sorting_indexes = np.array(sorting_indexes)

print(sorting_indexes.shape)

In [ ]:
print(unique_energy, unique_angle)

In [ ]:
grad_b_omega = np.array(metadata['gradb_omega'].iloc[sorting_indexes].array).reshape(energy_grid.shape)

In [ ]:
fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('vlag', as_cmap=True)

img = ax.imshow(grad_b_omega / (2 * np.pi), cmap=cmap, extent=(18575, 18600, 90, 84), aspect='auto')

fig.colorbar(img)

# remove grad-b shift, do beamforming, signal 1

In [ ]:
signal_energy = 18595
signal_angle = 89.5 

index = template_metadata[(template_metadata['energy'] == signal_energy) & (template_metadata['theta_min'] == signal_angle)].index[0]

signal = template_data.data[index, :].reshape((60, 8192 * 3))


ngrid = 21
x_pos_range = np.linspace(-0.01, -0.03, ngrid)
y_pos_range = np.linspace(-0.01, 0.01, ngrid)
grad_b_freq = 2.5e3

shifted_signal = GradBShift(signal, 0.02, 0.00, -grad_b_freq)
#shifted_signal = GradBShift(signal, 0.02, 0.00, grad_b_freq)

#pos_beam = np.linspace(-xpos_range, pos_range, ngrid)

sum_signal = ShiftAndSum(shifted_signal, x_pos_range, y_pos_range, 0)